In [1]:
'''
Resource Control/Monitor
- reference : https://docs.python.org/3/library/resource.html
- Note that this toy code is supposed to be run in CPU runtime.
'''
import resource  # 메모리 리소스 제한

## Limit memory

In [2]:
# See -1, -1 for the initial setting
print(resource.getrlimit(resource.RLIMIT_AS))              # print out in bytes

(-1, -1)


In [3]:
def memory_limit(ratio):
    soft, hard = resource.getrlimit(resource.RLIMIT_AS) # hard : 고정되서 넘어서는 안되는 메모리량, soft: 경고?
    resource.setrlimit(resource.RLIMIT_AS, (get_memory() * 1024 * ratio, hard)) # KB -> Byte

def get_memory():
    with open('/proc/meminfo', 'r') as mem:
        free_memory = 0
        for i in mem:
            sline = i.split()
            if str(sline[0]) in ('MemFree:', 'Buffers:', 'Cached:'):
                free_memory += int(sline[1]) # Free_memory = MemFree+Buffers+Cached
    return free_memory

In [4]:
# Check memory limit in MB
round(resource.getrlimit(resource.RLIMIT_AS)[0]/(1024*1024), 3) # resource.RLIMIT_AS returns in MB (ref: https://docs.python.org/3/library/resource.html)

-0.0

In [5]:
# Restrict memory_limit up to 80%
memory_limit(0.8) # 리미트 설정

In [6]:
# Check memory limit in MB
round(resource.getrlimit(resource.RLIMIT_AS)[0]/(1024*1024), 3) # resource.RLIMIT_AS returns in MB (ref: https://docs.python.org/3/library/resource.html)

9904.291

## Limit Core

In [7]:
# See -1, -1 for the initial setting
print(resource.getrlimit(resource.RLIMIT_CORE))

(-1, -1)


In [8]:
def core_limit(num):
    soft, hard = resource.getrlimit(resource.RLIMIT_CORE)
    resource.setrlimit(resource.RLIMIT_CORE, (num, hard))

In [9]:
core_limit(4) # 코어 제한

In [10]:
resource.getrlimit(resource.RLIMIT_CORE)

(4, -1)

## Mobilenet V2


<center><img src='https://gaussian37.github.io/assets/img/dl/concept/mobilenet_v2/1.png' width=50%></center>




In [11]:
import torch
import urllib

In [12]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
model.eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [17]:
# Download an example image from the pytorch website
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [18]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

In [19]:
with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# print(torch.nn.functional.softmax(output[0], dim=0))

## Memory Limit: 10%

In [25]:
# Restrict memory_limit up to 10%
memory_limit(0.1)

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# print(torch.nn.functional.softmax(output[0], dim=0))